In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import rc

rc('mathtext', default='regular')

# === Load data ===
root_data = '../../data/'
gw_all = pd.read_csv(root_data + 'cr2sub/cr2sub_v1.1_gwl_mon_clean.csv', parse_dates=['date'])
gw_all_att = pd.read_csv(root_data + 'cr2sub/cr2sub_v1.1_attributes.csv')
pr_camels = pd.read_csv(root_data + 'cr2met/cr2met_pr_v2.5_mon_1960_2025_camels_v2021.csv', parse_dates=['Index'])
pr_bna = pd.read_csv(root_data + 'cr2met/cr2met_pr_v2.5_mon_1960_2025_bna.csv', parse_dates=['Index'])
et_camels = pd.read_csv(root_data + 'cr2met/et_wb_cr2met_v2.5_cr2luc_mon_1960_2020_camels_v2021.csv', parse_dates=['date'])
et_bna = pd.read_csv(root_data + 'cr2met/et_wb_cr2met_v2.5_cr2luc_beta_mon_1960_2020_bna.csv', parse_dates=['date'])

# === Folder to store results ===
out_dir = os.path.join("../../data/ms_data")
os.makedirs(out_dir, exist_ok=True)

clean_codes = [col for col in gw_all.columns if col != 'date']

# === Global parameters ===
min_obs = 100

# Ensure string ids to match column names
gw_all_att['cr2sub_id'] = gw_all_att['cr2sub_id'].astype(str)

# === Pre-allocate ===
wells_pr_all = pd.DataFrame()
wells_et_all = pd.DataFrame()
wells_gw_all = pd.DataFrame()

for cod in clean_codes:
    gw = pd.Series(gw_all[cod].values, index=gw_all['date'])
    gw_ser = pd.Series(gw.values, index=gw.index, name=cod)
    gw_att_row = gw_all_att[gw_all_att['cr2sub_id'] == cod]

    if gw_att_row.empty:
        print(f"⚠️ Attributes not available for {cod}. Skipping.")
        continue

    gw_att = gw_att_row.iloc[0]

    if gw.notna().sum() < min_obs:
        continue  # Skip well
    
    id_camels = gw_att['cr2sub_in_basin_camels']
    id_bna = gw_att['cr2sub_in_basin_bna']

    camels_col = None
    if pd.notna(id_camels):
        camels_col = f"{int(id_camels):07d}"
        if camels_col not in pr_camels.columns or camels_col not in et_camels.columns:
            camels_col = None

    bna_col = None
    if pd.notna(id_bna):
        bna_col = f"{int(id_bna):03d}"
        if bna_col not in pr_bna.columns or bna_col not in et_bna.columns:
            bna_col = None

    if camels_col:
        pr_ser = pd.Series(pr_camels[camels_col].values, index=pr_camels['Index'], name=cod)
        et_ser = pd.Series(et_camels[camels_col].values, index=et_camels['date'], name=cod)
    elif bna_col:
        pr_ser = pd.Series(pr_bna[bna_col].values, index=pr_bna['Index'], name=cod)
        et_ser = pd.Series(et_bna[bna_col].values, index=et_bna['date'], name=cod)
    else:
        print(f"⚠️ Precip data not available for {cod}. Skipping.")
        continue

    # Align time span
    start = max(gw_ser.index.min(), pr_ser.index.min())
    end = min(gw_ser.index.max(), pr_ser.index.max())
    pr_ser = pr_ser.sort_index().loc[start:end]
    et_ser = et_ser.sort_index().loc[start:end]
    gw_ser = gw_ser.sort_index().loc[start:end]

    # Append to dataframe
    wells_pr_all = wells_pr_all.join(pr_ser, how='outer') if not wells_pr_all.empty else pr_ser.to_frame()
    wells_et_all = wells_et_all.join(et_ser, how='outer') if not wells_et_all.empty else et_ser.to_frame()
    wells_gw_all = wells_gw_all.join(gw_ser, how='outer') if not wells_gw_all.empty else gw_ser.to_frame()

# === Compute monthly anomalies (deseasonalize) ===
wells_gw_all_an = wells_gw_all - wells_gw_all.groupby(wells_gw_all.index.month).transform('mean')
wells_pr_all_an = wells_pr_all - wells_pr_all.groupby(wells_pr_all.index.month).transform('mean')
wells_et_all_an = wells_et_all - wells_et_all.groupby(wells_et_all.index.month).transform('mean')

# === Save results ===
if not wells_gw_all.empty:


    # Save raw series
    raw_to_save = [
        ("wells_gw.csv", wells_gw_all),
        ("wells_pr.csv", wells_pr_all),
        # ("wells_et.csv", wells_et_all),
    ]

    for fname, df in raw_to_save:
        df = df.sort_index()
        df.index.name = "date"
        df.to_csv(os.path.join(out_dir, fname))

    # Save anomalies
    an_to_save = [
        ("wells_gw_an.csv", wells_gw_all_an),
        ("wells_pr_an.csv", wells_pr_all_an),
        # ("wells_et_an.csv", wells_et_all_an),
    ]

    for fname, df in an_to_save:
        df = df.sort_index()
        df.index.name = "date"
        df.to_csv(os.path.join(out_dir, fname))

else:
    print("⚠️ No CAMELS/BNA wells collected — nothing to save.")


print("Finished!")


Finished!
